In [22]:
# pip install transformers

In [23]:
# !pip install datasets

In [39]:
import os
from datasets import Dataset
from sklearn.metrics import accuracy_score

from transformers import BertTokenizer
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import ast
import torch

In [40]:

from transformers import AutoModel, AutoConfig

# Path to the directory containing config.json and model.safetensors
model_directory = '../Models/Without_Nationality/won2_directory/'

# Load the configuration
config = AutoConfig.from_pretrained(model_directory)

# Load the model using the configuration
model_NN = AutoModel.from_pretrained(model_directory, config=config, use_safetensors=True)



Some weights of BertModel were not initialized from the model checkpoint at ../Models/Without_Nationality/won2_directory/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')

In [42]:
file_path = '../data/data_20_percent.csv'
df = pd.read_csv(file_path)

In [43]:
df

,wrong_sentence,error_index
0,معلم و شاگرد باید توجه داشته باشنند که اشتغالت...,"[0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,زباله در سطل است . تلفن با رایانه روی میز است ...,"[40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,این دِرخت بَرگ و میوه دارد [عراق],"[40, 40, 0, 0, 0, 0]"
3,من چند تا دوست دارم اما یکی از آنها خوبتر است ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,او میکوشد تا جهان خوب و دقیق را ببیند سخنان دی...,"[0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0,..."
...,...,...
1812,توضو و فروتنی در مقابل تکبّر و خود بزرگی بینی ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1813,فقط غذاها نیست مثلاً برای من خوابیدن هم خیلی م...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12,..."
1814,لذت های زندگی بسم الله الرحمن الرحیم موضوع : ا...,"[24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0..."
1815,چاپلوسان ثناگو را از خود دور بدار زیرا آن نقسا...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0,..."


In [44]:
# Correct `id2label` mapping
model_NN.config.id2label = {
    1: "negative",
    # 1: "MainSigns_Consonants",
    # 2: "MainSigns_Vowels",
    # 3: "DiacriticSigns_Tashdid",
    # 4: "DiacriticSigns_Madd",
    # 5: "DiacriticSigns_Tanwin",
    # 6: "Form_Boundary",
    # 7: "Form_AlifLam",
    # 8: "Form_Place",
    # 9: "Form_Dot",
    # 10: "Punctuation"
    0:"posetive"

}

# Correct `label2id` mapping
model_NN.config.label2id = {
    "negative": 1,
    # "MainSigns_Consonants": 1,
    # "MainSigns_Vowels": 2,
    # "DiacriticSigns_Tashdid": 3,
    # "DiacriticSigns_Madd": 4,
    # "DiacriticSigns_Tanwin": 5,
    # "Form_Boundary": 6,
    # "Form_AlifLam": 7,
    # "Form_Place": 8,
    # "Form_Dot": 9,
    # "Punctuation": 10
    "posetive":0
}


In [45]:
# Function to safely convert string to list
def safe_eval(value):
    import ast
    try:
        return ast.literal_eval(value)
    except:
        return []


df['error_index'] = df['error_index'].apply(safe_eval)

In [46]:
df['error_index'] =df['error_index'].apply(lambda lst: [1 if x!= 0 else x for x in lst])

In [47]:
df

,wrong_sentence,error_index
0,معلم و شاگرد باید توجه داشته باشنند که اشتغالت...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,زباله در سطل است . تلفن با رایانه روی میز است ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,این دِرخت بَرگ و میوه دارد [عراق],"[1, 1, 0, 0, 0, 0]"
3,من چند تا دوست دارم اما یکی از آنها خوبتر است ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,او میکوشد تا جهان خوب و دقیق را ببیند سخنان دی...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1812,توضو و فروتنی در مقابل تکبّر و خود بزرگی بینی ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1813,فقط غذاها نیست مثلاً برای من خوابیدن هم خیلی م...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1814,لذت های زندگی بسم الله الرحمن الرحیم موضوع : ا...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1815,چاپلوسان ثناگو را از خود دور بدار زیرا آن نقسا...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."


In [48]:
def predict_sentence(sentence):
    # Tokenize the input text
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)
    
    # Forward pass to get outputs
    outputs = model_NN(**inputs)

    # Get the last hidden state (representation for each token)
    last_hidden_state = outputs.last_hidden_state  # shape: (batch_size, seq_len, hidden_size)
    
    # Classification head: Define a dense layer for each token
    classification_head = torch.nn.Linear(last_hidden_state.shape[2], 1)  # Binary classification
    
    # Forward the last hidden state through the classification head
    token_logits = classification_head(last_hidden_state)  # shape: (batch_size, seq_len, 1)
    
    # Squeeze the logits to remove the last dimension
    token_logits = token_logits.squeeze(-1)  # shape: (batch_size, seq_len)
    
    # Sigmoid to convert logits to probabilities
    token_probabilities = torch.sigmoid(token_logits)
    
    # Get the predicted class for each token (0 or 1)
    token_predictions = (token_probabilities > 0.5).int()

    # Return only the prediction numbers as a list
    return token_predictions[0].tolist()

# Apply the function to each sentence in the DataFrame
df['prediction'] = df['wrong_sentence'].apply(predict_sentence)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [49]:
x=['d','e','w','s','e,','e','d,','s,','s']
c=['e','w','s']
print(x[1:len(c)])

['e', 'w']


In [50]:
def trim_predictions(predictions, error_index):
    # Drop the first item in the prediction list
    # Trim the prediction list based on the length of error_index
    error_index_length = len(error_index)
    return predictions[1:error_index_length+1]

# Apply trimming function to each row in the DataFrame
df['prediction'] = df.apply(lambda row: trim_predictions(row['prediction'], row['error_index']), axis=1)


In [51]:
df

,wrong_sentence,error_index,prediction
0,معلم و شاگرد باید توجه داشته باشنند که اشتغالت...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, ..."
1,زباله در سطل است . تلفن با رایانه روی میز است ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,این دِرخت بَرگ و میوه دارد [عراق],"[1, 1, 0, 0, 0, 0]","[0, 1, 1, 0, 0, 0]"
3,من چند تا دوست دارم اما یکی از آنها خوبتر است ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,او میکوشد تا جهان خوب و دقیق را ببیند سخنان دی...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
1812,توضو و فروتنی در مقابل تکبّر و خود بزرگی بینی ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, ..."
1813,فقط غذاها نیست مثلاً برای من خوابیدن هم خیلی م...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1814,لذت های زندگی بسم الله الرحمن الرحیم موضوع : ا...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1815,چاپلوسان ثناگو را از خود دور بدار زیرا آن نقسا...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [52]:
def compute_accuracy(row):
    # Extract trimmed predictions and correct labels
    y_pred = row['prediction']
    y_true = row['error_index']
    
    # Debugging output
    # print(f"True Labels: {len(y_true)}")
    # print(f"Predictions: {len(y_pred)}")
    
    # Check if lengths match
    if len(y_true) != len(y_pred):
        print(f"True Labels: {y_true}")
        print(f"Predictions: {y_pred}")
        print(len(row['wrong_sentence']))
        y_true=y_true[:len(y_pred)]
        # raise ValueError(f"Inconsistent lengths: y_true length {len(y_true)}, y_pred length {len(y_pred)}")
            
    # Compute accuracy score
    return accuracy_score(y_true, y_pred)

# Compute accuracy for each row
df['accuracy'] = df.apply(compute_accuracy, axis=1)

# Calculate the overall accuracy
overall_accuracy = df['accuracy'].mean()

# Print the DataFrame with accuracy
# print(df)

# Print overall accuracy
print(f"Overall Accuracy: {overall_accuracy:.2f}")

True Labels: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
119
True Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Predictions: [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
191
True Labels: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Predictions: [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
171
True Labels: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Predictions: [0, 0, 0, 0, 0, 

In [ ]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# import torch



# # Sample input text
# text = "دیوارتلتا  عغلا ب کلاس سفد است[ایران]"


# # Tokenize the input text
# inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# # Forward pass to get outputs
# outputs = model_NN(**inputs)

# # Get the last hidden state (representation for each token)
# last_hidden_state = outputs.last_hidden_state  # shape: (batch_size, seq_len, hidden_size)

# # Classification head: Define a dense layer for each token
# # Assuming binary classification for each token (you can change this for more classes)
# classification_head = torch.nn.Linear(last_hidden_state.shape[2], 1)  # Output size is 1 for binary classification
# print(last_hidden_state.shape)
# # Forward the last hidden state through the classification head
# token_logits = classification_head(last_hidden_state)  # shape: (batch_size, seq_len, 1)
# print(token_logits.shape)
# # Squeeze the logits to remove the last dimension
# token_logits = token_logits.squeeze(-1)  # shape: (batch_size, seq_len)
# print(token_logits)
# # Sigmoid to convert logits to probabilities (for binary classification)
# token_probabilities = torch.sigmoid(token_logits)
# print(token_probabilities)
# # Get the predicted class for each token (0 or 1)
# token_predictions = (token_probabilities > 0.5).int()

# # Decode token ids to tokens
# tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# # Print each token with its predicted label
# for token, prediction in zip(tokens, token_predictions[0]):
#     print(f"Token: {token}, Predicted class: {prediction.item()}")


In [ ]:
from datasets import load_from_disk

# Load the dataset from the directory
test_dataset = load_from_disk('../data/test_dataset_directory')


In [ ]:

def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Determine the length of the longest sequence in the batch
    max_len = max(len(ids) for ids in input_ids)

    # Pad input_ids and attention_masks to the length of the longest sequence
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)

In [ ]:
import torch

# Set the format to PyTorch tensors
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [ ]:
# custom_test_dataset['input_ids']

In [ ]:

     
import torch
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    # Extract the input_ids, attention_mask, and labels from the batch
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]

    # Pad the sequences to the length of the longest sequence in the batch
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=0)

    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'labels': labels_padded
    }

# Use the custom collate function in your DataLoader
from torch.utils.data import DataLoader



In [ ]:

# Create a DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Assuming binary classification, and using a classification head
classification_head = torch.nn.Linear(model_NN.config.hidden_size, 1)  # Hidden size is the size of the last hidden state

model_NN.eval()  # Set the model to evaluation mode

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Get the model outputs
        outputs = model_NN(input_ids=input_ids, attention_mask=attention_mask)

        # Extract the last hidden state (token representations)
        last_hidden_state = outputs.last_hidden_state  # shape: (batch_size, seq_len, hidden_size)

        # Forward the last hidden state through the classification head for each token
        token_logits = classification_head(last_hidden_state)  # shape: (batch_size, seq_len, 1)

        # Squeeze the logits to remove the last dimension
        token_logits = token_logits.squeeze(-1)  # shape: (batch_size, seq_len)

        # Apply sigmoid to convert logits to probabilities
        token_probabilities = torch.sigmoid(token_logits)  # shape: (batch_size, seq_len)

        # Get the predicted class for each token (0 or 1)
        token_predictions = (token_probabilities > 0.5).int()  # shape: (batch_size, seq_len)

        # Flatten predictions and labels for accuracy calculation
        all_preds.extend(token_predictions.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy().flatten())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

# Optionally, decode and print token predictions for the first batch
tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())

# Print each token with its predicted label
for token, prediction in zip(tokens, token_predictions[0].cpu().numpy()):
    print(f"Token: {token}, Predicted class: {prediction}")
